## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [51]:

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio
        self.tipo = tipo
        self._cantidad = cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor > 0:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if valor > 0:
            self._cantidad = valor

    def valorTotal(self):
        return self._precio * self._cantidad

In [52]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [53]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo if codigo.isdigit() and len(codigo) == 4 else '0000'
        self.nombre = nombre if 1 <= len(nombre) <= 100 else 'Producto sin nombre'
        self.tipo = tipo if 0 <= len(tipo) <= 20 else ''
        self._precio = self.validar_precio(precio)
        self._cantidad = self.validar_cantidad(cantidad)

    @staticmethod
    def validar_precio(precio):
        if 10 <= precio <= 10000:
            return precio
        return 10

    @staticmethod
    def validar_cantidad(cantidad):
        if 0 <= cantidad <= 1000:
            return cantidad
        return 0

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, nuevo_precio):
        self._precio = self.validar_precio(nuevo_precio)

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, nueva_cantidad):
        self._cantidad = self.validar_cantidad(nueva_cantidad)

    def valorTotal(self):
        return self.precio * self.cantidad


class Oferta:
    def __init__(self, descripcion="No hay descripción", codigos=None, tipos=None):
        self.descripcion = descripcion if 1 <= len(descripcion) <= 100 else "No hay descripción"
        self.codigos = codigos if codigos is not None else []  
        self.tipos = tipos if tipos is not None else []

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos


class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="No hay descripción", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)
        self.descuento = max(0, min(descuento, 100)) 

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return producto.precio * cantidad * self.descuento / 100
        return 0


class Oferta2x1(Oferta):
    def __init__(self, descripcion="No hay descripcion", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (cantidad // 2) * producto.precio
        return 0




In [54]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [55]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                producto = Producto(
                    row['codigo'], row['nombre'], int(row['precio']), row['tipo'], int(row['cantidad'])
                )
                catalogo.agregar(producto)
        return catalogo

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='') as file:
            fieldnames = ['codigo', 'nombre', 'precio', 'tipo', 'cantidad']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for producto in self.productos:
                writer.writerow({
                    'codigo': producto.codigo,
                    'nombre': producto.nombre,
                    'precio': producto.precio,
                    'tipo': producto.tipo,
                    'cantidad': producto.cantidad
                })

    def agregar(self, producto):
        self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.aplica(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        return oferta.calcularDescuento(producto, cantidad) if oferta else 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def vender(self, producto, cantidad):
        producto_encontrado = self.buscar(producto.codigo)
        if producto_encontrado and producto_encontrado.cantidad >= cantidad:
            producto_encontrado.cantidad -= cantidad

    def informe(self):
        tipos_productos = set(producto.tipo for producto in self.productos)
        informe = (
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {self.cantidadUnidades}\n"
            f"Precio Promedio: {self.valorTotal / self.cantidadUnidades:.2f}\n"
            f"Valor total: {self.valorTotal:.2f}\n"
            f"Tipos de productos: {', '.join(tipos_productos)}\n"
            f"Ofertas:\n"
        )
        for oferta in self.ofertas:
            informe += f"- {oferta.descripcion()}\n"
        return informe

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

class Oferta:
    def aplica(self, producto, cantidad):
        raise NotImplementedError

    def calcularDescuento(self, producto, cantidad):
        raise NotImplementedError

class Oferta2x1(Oferta):
    def __init__(self, tipos):
        self.tipos = tipos

    def aplica(self, producto, cantidad):
        return producto.tipo in self.tipos and cantidad >= 2

    def calcularDescuento(self, producto, cantidad):
        return (cantidad // 2) * producto.precio

    def descripcion(self):
        return "Oferta 2x1"

class OfertaDescuento(Oferta):
    def __init__(self, porcentaje, codigos):
        self.porcentaje = porcentaje
        self.codigos = codigos

    def aplica(self, producto, cantidad):
        return producto.codigo in self.codigos

    def calcularDescuento(self, producto, cantidad):
        return (producto.precio * cantidad) * (self.porcentaje / 100)

    def descripcion(self):
        return f"Oferta {self.porcentaje}% de descuento"


In [56]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [57]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre 
        self.cuit = cuit      

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if 1 <= len(value) <= 100:
            self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        if self.validar_cuit(value):
            self._cuit = value

    @staticmethod
    def validar_cuit(cuit):
        return re.match(r'^\d{2}-\d{8}-\d$', cuit) is not None


In [58]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [59]:
from datetime import datetime

class Factura:
    _ultimo_numero = 100  # Número inicial, puede ser ajustado

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime("%Y-%m-%d")
        self.items = []  # Lista de tuplas (producto, cantidad, oferta_aplicada, descuento)

    def agregar(self, producto, cantidad):
        if not isinstance(cantidad, int) or cantidad <= 0:
            raise ValueError("La cantidad debe ser un entero positivo.")

        # Verificar si el producto existe en el catálogo
        if not self.catalogo.buscar(producto.codigo):
            raise ValueError(f"El producto con código {producto.codigo} no está en el catálogo.")

        # Verificar si hay suficiente cantidad en el catálogo
        if producto.cantidad < cantidad:
            raise ValueError(f"No hay suficiente cantidad de {producto.nombre} en existencia.")

        # Buscar oferta aplicable
        oferta = self.catalogo.buscarOferta(producto, cantidad)
        descuento = oferta.calcularDescuento(producto, cantidad) if oferta else 0
        descripcion_oferta = oferta.descripcion if oferta else ""

        # Agregar o actualizar el item en la factura
        item_existente = next((item for item in self.items if item[0].codigo == producto.codigo), None)
        if item_existente:
            # Actualizar cantidad y descuento
            nueva_cantidad = item_existente[1] + cantidad
            nueva_oferta = self.catalogo.buscarOferta(producto, nueva_cantidad)
            nueva_descuento = nueva_oferta.calcularDescuento(producto, nueva_cantidad) if nueva_oferta else 0
            nueva_descripcion_oferta = nueva_oferta.descripcion if nueva_oferta else ""
            # Actualizar el item en la lista
            self.items = [
                (item[0], nueva_cantidad, nueva_descripcion_oferta, nueva_descuento)
                if item[0].codigo == producto.codigo else item for item in self.items
            ]
        else:
            # Si el producto no está aún en la factura, agregarlo
            self.items.append((producto, cantidad, descripcion_oferta, descuento))

        # Vender el producto (actualizar el catálogo)
        self.catalogo.vender(producto, cantidad)

    @property
    def cantidadProductos(self):
        return len(self.items)

    @property
    def cantidadUnidades(self):
        return sum(item[1] for item in self.items)

    @property
    def subtotal(self):
        return sum(item[0].precio * item[1] for item in self.items)

    @property
    def descuentos(self):
        return sum(item[3] for item in self.items)

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        lines = []
        lines.append(f"Factura: {self.numero}")
        lines.append(f"Fecha  : {self.fecha}")
        lines.append(f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n")

        for item in self.items:
            producto, cantidad, descripcion_oferta, descuento = item
            subtotal_item = producto.precio * cantidad
            line = f"- {cantidad} {producto.nombre:<20} x ${producto.precio:.2f} = ${subtotal_item:.2f}"
            lines.append(line)
            if descripcion_oferta and descuento > 0:
                # Indentar la descripción de la oferta
                lines.append(f"      {descripcion_oferta:<40} - ${descuento:.2f}")

        lines.append("\n" + f"{'Subtotal:':>50} ${self.subtotal:.2f}")
        lines.append(f"{'Descuentos:':>50} -${self.descuentos:.2f}")
        lines.append(f"{'-'*50}")
        lines.append(f"{'TOTAL:':>50} ${self.total:.2f}")

        return "\n".join(lines)

class Producto:
    def __init__(self, codigo, nombre, precio, categoria, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.categoria = categoria
        self.cantidad = cantidad

class Catalogo:
    def __init__(self):
        self.productos = {}
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos[producto.codigo] = producto

    def buscar(self, codigo):
        return self.productos.get(codigo)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def vender(self, producto, cantidad):
        if producto.codigo in self.productos:
            self.productos[producto.codigo].cantidad -= cantidad

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, 'r') as file:
            next(file)  # Omitir la primera línea si es un encabezado
            for linea in file:
                # Suponiendo que el archivo tiene el siguiente formato CSV:
                # codigo,nombre,precio,categoria,cantidad
                codigo, nombre, precio, categoria, cantidad = linea.strip().split(',')
                producto = Producto(codigo, nombre, float(precio), categoria, int(cantidad))
                catalogo.agregar(producto)
        return catalogo

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos.values())

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos.values())

class Oferta2x1:
    def __init__(self, tipos):
        self.tipos = tipos
        self.descripcion = "Oferta 2x1"

    def esAplicable(self, producto, cantidad):
        return producto.categoria in self.tipos and cantidad >= 2

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (cantidad // 2) * producto.precio
        return 0

class OfertaDescuento:
    def __init__(self, porcentaje, codigos):
        self.porcentaje = porcentaje
        self.codigos = codigos
        self.descripcion = f"Descuento {porcentaje}%"

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (producto.precio * cantidad) * (self.porcentaje / 100)
        return 0

# Cliente para prueba
class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit


In [60]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [62]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

TypeError: Oferta2x1.__init__() got an unexpected keyword argument 'codigos'